In [1]:
import pandas as pd
import seaborn as sbs
import numpy as np
import calendar
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn import neural_network
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor

In [2]:
train  = pd.read_csv('tp_bike/trip_train.csv')
test = pd.read_csv('tp_bike/trip_test.csv')
trip = pd.read_csv('tp_bike/trip.csv')

In [3]:
del train['end_date']
del train['end_station_name']
del train['end_station_id']
del test['end_date']
del test['end_station_name']
del test['end_station_id']

In [4]:
fechaYhora = train['start_date'].str.split(' ')
fecha = fechaYhora.map(lambda x: x[0])
hora = fechaYhora.map(lambda x: x[1])
train = train.assign(date_start = fecha, time_start = hora)

In [5]:
train['date_start'] = pd.to_datetime(train['date_start'])
train['weekday'] = train['date_start'].map(lambda x: x.weekday_name)
train['hour_start'] = pd.to_datetime(train['time_start'], format = '%H:%M', errors = 'coerce').dt.hour

In [6]:
fechaYhora = test['start_date'].str.split(' ')
fecha = fechaYhora.map(lambda x: x[0])
hora = fechaYhora.map(lambda x: x[1])
test = test.assign(date_start = fecha, time_start = hora)

test['date_start'] = pd.to_datetime(test['date_start'])
test['weekday'] = test['date_start'].map(lambda x: x.weekday_name)
test['hour_start'] = pd.to_datetime(test['time_start'], format = '%H:%M', errors = 'coerce').dt.hour

In [7]:
train['subscription_type'] = train.subscription_type.map({'Subscriber':1, 'Customer':0})
train['weekday'] = train.weekday.map({'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4,\
                                      'Friday': 5, 'Saturday': 6, 'Sunday': 7})

In [8]:
test['weekday'] = test.weekday.map({'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7})
test['subscription_type'] = test.subscription_type.map({'Subscriber':1, 'Customer':0})

In [9]:
train['month_start'] = train['date_start'].map(lambda x: x.month)
test['month_start'] = test['date_start'].map(lambda x: x.month)

In [10]:
train.zip_code = train.zip_code.str.replace('nil','')
train.zip_code = pd.to_numeric(train.zip_code,errors='coerce')
test.zip_code = test.zip_code.str.replace('nil','')
test.zip_code = pd.to_numeric(test.zip_code,errors='coerce')

print "fin de Pre-procesamiento"

fin de Pre-procesamiento


In [11]:
cols = ['start_station_id', 'subscription_type', 'month_start', 'weekday', 'hour_start']
features = train[cols]
duration = train.duration

In [12]:
tripIdDuration = trip[['id','duration']]
target = test[['id']]
testTarget = pd.merge(tripIdDuration,target, how = "inner", on = "id")
duration_real = testTarget['duration']
test_reg = test[cols]

In [13]:
#Linear Regression
reg = linear_model.LinearRegression()

x_train, x_test, y_train, y_test = train_test_split(features, duration, test_size = 0.75, random_state = 24)

reg.fit(x_train, y_train) 

prediction_reg = reg.predict(test_reg)

print "Linear Regression aplicado"

Linear Regression aplicado


In [14]:
#Random Forest Regression
rfr = RandomForestRegressor(n_estimators = 100, min_samples_leaf = 2, random_state = 0)

x1_train, x1_test, y1_train, y1_test = train_test_split(features, duration, test_size = 0.75, random_state = 24)

rfr.fit(x1_train,y1_train)

prediction_rfr = rfr.predict(test_reg)

print "Random Forest aplicado"

Random Forest aplicado


In [15]:
#Gradient Boosting Regression
gbr = GradientBoostingRegressor()

x2_train, x2_test, y2_train, y2_test = train_test_split(features, duration, test_size = 0.75, random_state = 24)

gbr.fit(x2_train,y2_train)

prediction_gbr = gbr.predict(test_reg)

print "Gradient Boosting aplicado"

Gradient Boosting aplicado


In [16]:
#Combino las Predicciones que obtuve
prediccion_avg = []

for pos in range(0, len(prediction_reg)):
    
    mean = round(float(prediction_reg[pos] + prediction_gbr[pos] + prediction_rfr[pos])/3,8)
    prediccion_avg.append(mean)

score_avg = mean_squared_error(prediccion_avg, duration_real)
score_avg

45090265.45089709

In [17]:
ids = test.id
data = {'id': ids, 'duration': prediccion_avg}
submitFinal = pd.DataFrame(data)
submitFinal.to_csv('SubmitFinal.csv',index=False)